In [1]:
import torch
import torch.nn as nn
import yaml
import glob
import scipy.io as scio
import random
import numpy as np
import os
import csv
from tqdm import tqdm
from evaluate import error
from mmfi import make_dataset, make_dataloader
from lidar_point_transformer import PointTransformerReg

In [2]:
dataset_root = '/media/xinyan/My Passport/FYP/Data'
with open('config.yaml', 'r') as fd:
    config = yaml.load(fd, Loader=yaml.FullLoader)

train_dataset, val_dataset = make_dataset(dataset_root, config)

S01 ['A01', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A15', 'A16', 'A24', 'A25', 'A26']
S02 ['A01', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A15', 'A16', 'A24', 'A25', 'A26']
S03 ['A01', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A15', 'A16', 'A24', 'A25', 'A26']
S04 ['A01', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A15', 'A16', 'A24', 'A25', 'A26']
S05 ['A01', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A15', 'A16', 'A24', 'A25', 'A26']
S06 ['A01', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A15', 'A16', 'A24', 'A25', 'A26']
S07 ['A01', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A15', 'A16', 'A24', 'A25', 'A26']
S08 ['A01', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A15', 'A16', 'A24', 'A25', 'A26']
S09 ['A01', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A15', 'A16', 'A24', 'A25', 'A26']
S10 ['A01', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A15', 'A16', 'A24', 'A25', 'A26']
S11 ['A01', 'A06', 'A07', 'A08', 'A09', 

In [3]:
def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.
    '''
    ## get sequence lengths
    # for t in batch:
        # print(t['output'].type)
    #     print(a)
        # print(t[0].shape,t[1].shape)
    info = []
    for t in batch:
        info.append([t['subject'], t['action'],t['idx']])
    kpts = []
    [kpts.append(np.array(t['output'])) for t in batch]
    kpts = torch.FloatTensor(np.array(kpts))

    lengths = torch.tensor([t['input_lidar'].shape[0] for t in batch ])
    ## padd
    batch = [torch.Tensor(np.array(t['input_lidar'])) for t in batch ]
    batch = torch.nn.utils.rnn.pad_sequence(batch)
    ## compute mask
    batch = batch.permute(1,0,2)
    mask = (batch != 0)

    return batch, kpts, lengths, mask, info

In [4]:
rng_generator = torch.manual_seed(config['init_rand_seed'])
train_loader = make_dataloader(train_dataset, is_training=True, generator=rng_generator, **config['loader'], collate_fn = collate_fn_padd)
val_loader = make_dataloader(val_dataset, is_training=False, generator=rng_generator, **config['loader'], collate_fn = collate_fn_padd)

In [5]:
for i,data in enumerate(val_loader):
    inputs, labels, length, mask, info = data
    print(inputs.shape)
    print(labels.shape)
    print(info)
    if i == 10:
        break

torch.Size([32, 1308, 3])
torch.Size([32, 17, 3])
[['S31', 'A01', 0], ['S31', 'A01', 1], ['S31', 'A01', 2], ['S31', 'A01', 3], ['S31', 'A01', 4], ['S31', 'A01', 5], ['S31', 'A01', 6], ['S31', 'A01', 7], ['S31', 'A01', 8], ['S31', 'A01', 9], ['S31', 'A01', 10], ['S31', 'A01', 11], ['S31', 'A01', 12], ['S31', 'A01', 13], ['S31', 'A01', 14], ['S31', 'A01', 15], ['S31', 'A01', 16], ['S31', 'A01', 17], ['S31', 'A01', 18], ['S31', 'A01', 19], ['S31', 'A01', 20], ['S31', 'A01', 21], ['S31', 'A01', 22], ['S31', 'A01', 23], ['S31', 'A01', 24], ['S31', 'A01', 25], ['S31', 'A01', 26], ['S31', 'A01', 27], ['S31', 'A01', 28], ['S31', 'A01', 29], ['S31', 'A01', 30], ['S31', 'A01', 31]]
torch.Size([32, 1307, 3])
torch.Size([32, 17, 3])
[['S31', 'A01', 32], ['S31', 'A01', 33], ['S31', 'A01', 34], ['S31', 'A01', 35], ['S31', 'A01', 36], ['S31', 'A01', 37], ['S31', 'A01', 38], ['S31', 'A01', 39], ['S31', 'A01', 40], ['S31', 'A01', 41], ['S31', 'A01', 42], ['S31', 'A01', 43], ['S31', 'A01', 44], ['S31', 

In [21]:
with open('model_config.yaml', 'r') as fd:
        model_cfg = yaml.load(fd, Loader=yaml.FullLoader)
lidar_cfg = model_cfg['lidar']
model = PointTransformerReg(lidar_cfg)

In [22]:
def test(model, tensor_loader, criterion1, criterion2, device):
    model.eval()
    test_mpjpe = 0
    test_pampjpe = 0
    test_mse = 0
    output_dict = {}
    for data in tqdm(tensor_loader):
        inputs, labels, _, _, info = data
        inputs = inputs.to(device)
        labels.to(device)
        labels = labels.type(torch.FloatTensor)
        outputs = model(inputs)
        outputs = outputs.type(torch.FloatTensor)
        outputs.to(device)
        test_mse += criterion1(outputs,labels).item() * inputs.size(0)

        outputs = outputs.detach().numpy()
        for i in range(len(outputs)):
            info_name = info[i][0]+'_'+info[i][1]
            if info_name not in output_dict:
                # print(info_name)
                output_dict[info_name] = np.zeros((297,17,3))
            output_dict[info_name][info[i][2],:,:] = outputs[i]
        labels = labels.detach().numpy()
        
        mpjpe, pampjpe = criterion2(outputs,labels)
        test_mpjpe += mpjpe.item() * inputs.size(0)
        test_pampjpe += pampjpe.item() * inputs.size(0)
    test_mpjpe = test_mpjpe/len(tensor_loader.dataset)
    test_pampjpe = test_pampjpe/len(tensor_loader.dataset)
    test_mse = test_mse/len(tensor_loader.dataset)
    print("mse: {:.8f}, mpjpe: {:.8f}, pampjpe: {:.8f}".format(float(test_mse), float(test_mpjpe),float(test_pampjpe)))
    return test_mpjpe, output_dict

In [23]:
train_criterion = nn.MSELoss()
test_criterion = error
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load('./pre-train_weights/lidar_p2_cross_scene.pt',map_location='cuda:0'))
model.to(device)
_,output_dict = test(model=model, tensor_loader= val_loader, criterion1 = train_criterion, criterion2 = test_criterion, device=device)

100%|██████████| 1207/1207 [05:29<00:00,  3.66it/s]

mse: 0.01486665, mpjpe: 0.18254361, pampjpe: 0.10235050


In [24]:
# for key in output_dict.keys():
#     print(key)
print(output_dict.keys())
predicted = output_dict['S35_A01']
print(predicted.shape)
gt = np.load('/media/xinyan/My Passport/FYP/Data/E04/S35/A01/ground_truth.npy')
print(gt.shape)
mpjpe, pampjpe = error(predicted, gt)
print(mpjpe, pampjpe)

dict_keys(['S31_A01', 'S31_A06', 'S31_A07', 'S31_A08', 'S31_A09', 'S31_A10', 'S31_A11', 'S31_A12', 'S31_A15', 'S31_A16', 'S31_A24', 'S31_A25', 'S31_A26', 'S32_A01', 'S32_A06', 'S32_A07', 'S32_A08', 'S32_A09', 'S32_A10', 'S32_A11', 'S32_A12', 'S32_A15', 'S32_A16', 'S32_A24', 'S32_A25', 'S32_A26', 'S33_A01', 'S33_A06', 'S33_A07', 'S33_A08', 'S33_A09', 'S33_A10', 'S33_A11', 'S33_A12', 'S33_A15', 'S33_A16', 'S33_A24', 'S33_A25', 'S33_A26', 'S34_A01', 'S34_A06', 'S34_A07', 'S34_A08', 'S34_A09', 'S34_A10', 'S34_A11', 'S34_A12', 'S34_A15', 'S34_A16', 'S34_A24', 'S34_A25', 'S34_A26', 'S35_A01', 'S35_A06', 'S35_A07', 'S35_A08', 'S35_A09', 'S35_A10', 'S35_A11', 'S35_A12', 'S35_A15', 'S35_A16', 'S35_A24', 'S35_A25', 'S35_A26', 'S36_A01', 'S36_A06', 'S36_A07', 'S36_A08', 'S36_A09', 'S36_A10', 'S36_A11', 'S36_A12', 'S36_A15', 'S36_A16', 'S36_A24', 'S36_A25', 'S36_A26', 'S37_A01', 'S37_A06', 'S37_A07', 'S37_A08', 'S37_A09', 'S37_A10', 'S37_A11', 'S37_A12', 'S37_A15', 'S37_A16', 'S37_A24', 'S37_A25',

In [27]:
save_root = '/media/xinyan/My Passport/FYP/Data/MMFi_Evaluation'
for key in output_dict.keys():
    npz = output_dict[key]
    save_dir = os.path.join(save_root, config['protocol'], config['split_to_use'], config['modality'], 'round1')
    os.makedirs(save_dir, exist_ok=True)
    sample_name = key + '.npz'
    # print(save_dir)
    np.savez(os.path.join(save_dir, sample_name), joints=npz)

/media/xinyan/My Passport/FYP/Data/MMFi_Evaluation/protocol2/cross_scene_split/lidar/round1
/media/xinyan/My Passport/FYP/Data/MMFi_Evaluation/protocol2/cross_scene_split/lidar/round1
/media/xinyan/My Passport/FYP/Data/MMFi_Evaluation/protocol2/cross_scene_split/lidar/round1
/media/xinyan/My Passport/FYP/Data/MMFi_Evaluation/protocol2/cross_scene_split/lidar/round1
/media/xinyan/My Passport/FYP/Data/MMFi_Evaluation/protocol2/cross_scene_split/lidar/round1
/media/xinyan/My Passport/FYP/Data/MMFi_Evaluation/protocol2/cross_scene_split/lidar/round1
/media/xinyan/My Passport/FYP/Data/MMFi_Evaluation/protocol2/cross_scene_split/lidar/round1
/media/xinyan/My Passport/FYP/Data/MMFi_Evaluation/protocol2/cross_scene_split/lidar/round1
/media/xinyan/My Passport/FYP/Data/MMFi_Evaluation/protocol2/cross_scene_split/lidar/round1
/media/xinyan/My Passport/FYP/Data/MMFi_Evaluation/protocol2/cross_scene_split/lidar/round1
/media/xinyan/My Passport/FYP/Data/MMFi_Evaluation/protocol2/cross_scene_split/l